In [4]:
import pandas as pd
from tiingo import TiingoClient
from ta.momentum import RSIIndicator, stoch
from ta.trend import MACD as md
from ta.volatility import AverageTrueRange as atr

In [243]:
client = TiingoClient({'api_key' : '7fb8e25ac6335f846ae82d3f3a28ed535f313592'})

df = pd.read_csv(R"X:\GitHub\beatingENd\BeatingEnDUS\Own Screener\IBD50.csv")

# datalist = (df['Ticker'].to_list())
datalist = ['NFLX', 'AAPL', 'GOOG']

data = client.get_dataframe(datalist,
                                    metric_name='adjClose',
                                      frequency='daily',
                                      startDate='2015-02-02',
                                      endDate='2020-10-10')

In [244]:
def RSI_base(data,ticker):
    MACD = md(close=data[ticker])
    data["MACD_diff"] = MACD.macd_diff()
    RSI = RSIIndicator(close=data[ticker])
    data["RSI"] = RSI.rsi()
    data["Buy_Signal"] = 0
    data["Pos"] = 0
    data["Sell_Signal"] = 0
    data["SL"] = 0 
    data["TP"] = 0
    data["TW"] = 0.0

    data = data.dropna()

    for i in range(0,len(data)):
        if data["Pos"][i-1] == 0:
            if data["RSI"].iloc[i] > 50 and data["MACD_diff"][i] > 0 and (data["Pos"][i] == 0):
                data["Buy_Signal"][i] = 1
                data["Pos"][i] = 1
                data["SL"][i] = (data[ticker][i]*0.97)
                data["TP"][i] = (data[ticker][i]*1.5)
            else:
                data["Buy_Signal"][i] = 0 
                data["Pos"][i] = 0

        elif data["Pos"][i-1] == 1:
            data["SL"][i] = data["SL"][i-1] 
            data["TP"][i] = data["TP"][i-1]
            if (data["NFLX"][i] > data["SL"][i-1]) or (data[ticker][i] < data["TP"][i-1]):
                data["Pos"][i] = 1
            if (data[ticker][i] < data["SL"][i]) or (data[ticker][i] > data["TP"][i-1]):
                data["Pos"][i] = 0
                data["Sell_Signal"][i] = 1

    for i in range(len(data)):
        if (data["Pos"][i] == 1): 
            data["TW"][i] = 1
        else:
            data["TW"][i] = 0

    chumps = data["TW"]

    return chumps

In [245]:
df = pd.DataFrame()

for i in datalist:
    temp = RSI_base(data,i)
    temp = temp.rename(i)
    df = pd.concat([df,temp],axis=1)

In [246]:
df

,NFLX,AAPL,GOOG
2015-03-20 00:00:00+00:00,0.00000,0.00000,0.00000
2015-03-23 00:00:00+00:00,0.00000,0.00000,0.00000
2015-03-24 00:00:00+00:00,0.00000,0.00000,0.00000
2015-03-25 00:00:00+00:00,0.00000,0.00000,0.00000
2015-03-26 00:00:00+00:00,0.00000,0.00000,0.00000
...,...,...,...
2020-10-05 00:00:00+00:00,1.00000,1.00000,0.00000
2020-10-06 00:00:00+00:00,1.00000,1.00000,0.00000
2020-10-07 00:00:00+00:00,1.00000,1.00000,0.00000
2020-10-08 00:00:00+00:00,1.00000,1.00000,0.00000


In [247]:
import bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [248]:
crosses = bt.Strategy('RSI', [bt.algos.WeighTarget(df),
                                    bt.algos.Rebalance()])


long_only = bt.Strategy('Benchmark', [bt.algos.RunOnce(),
                        bt.algos.SelectAll(),
                        bt.algos.WeighEqually(),
                        bt.algos.Rebalance()])

t = bt.Backtest(crosses,data)
s = bt.Backtest(long_only, data)

In [251]:
report = bt.run(t,s)
report.plot()
plt.title("Portfolio Returns")
plt.savefig("arg.png")

In [250]:
report.set_riskfree_rate(0.001)
report.display()

Stat                 RSI         Benchmark
-------------------  ----------  -----------
Start                2015-02-01  2015-02-01
End                  2020-10-09  2020-10-09
Risk-free rate       0.10%       0.10%

Total Return         2184.15%    424.94%
Daily Sharpe         1.14        1.14
Daily Sortino        1.83        1.88
CAGR                 73.36%      33.86%
Max Drawdown         -65.68%     -34.69%
Calmar Ratio         1.12        0.98

MTD                  8.41%       5.03%
3m                   5.17%       9.01%
6m                   99.95%      48.03%
YTD                  34.18%      52.05%
1Y                   109.80%     83.20%
3Y (ann.)            75.84%      35.32%
5Y (ann.)            67.93%      31.69%
10Y (ann.)           73.36%      33.86%
Since Incep. (ann.)  73.36%      33.86%

Daily Sharpe         1.14        1.14
Daily Sortino        1.83        1.88
Daily Mean (ann.)    79.85%      33.41%
Daily Vol (ann.)     69.90%      29.17%
Daily Skew           -0.30      